<pre>
          ) ) )                           ( ( (
        ( ( (                               ) ) )
      ) ) )                                  ( ( (
   (~~~~~~~~~)                            (~~~~~~~~~)
    | POWER |                              | REWOP |
    | HOUSE |                              | ESUOH |   
    I      _._                            _._      I    
    I    /'   `\                        /`   '\    I    
    I   |   N   |                      |   N   |   I    
    f   |   |~~~~~~~~~~~~~~||~~~~~~~~~~~~~~|   |   f    
  .'    |   ||~~~~~~~~|    ||    |~~~~~~~~||   |    '.  
/'______|___||__###___|____||____|___###__||___|______'\


MITO
MOTI lity
quantifier

</pre>

To run the motility quantifier:
1. Run the first code cell to select your experiment.
2. Change the CHANNEL variable in the second code cell to the name of the channel that you're analyzing.
3. Run the second code cell, wait for it to finish, and 

In [ ]:
%run utils/experiment_chooser.ipynb
experiment_chooser.choose()

In [ ]:
from improc.processes.motility import calc_motility
from multiprocessing import Pool
from skimage.exposure import rescale_intensity
import tifffile
import numpy as np
import pathlib
import csv
from tqdm import tqdm
from pystackreg import StackReg
from collections import defaultdict

### STACK ###

CHANNEL = "Cy5"

experiment = experiment_chooser.fetch()
experiment_base = experiment.experiment_dir
scratch_output = experiment.scratch_dir

if not scratch_output.exists():
    scratch_output.mkdir()
    
stacked_output = scratch_output / "stacked"
if not stacked_output.exists():
    stacked_output.mkdir()

def key(path):
    _, _, idx = path.name.split(".")[0].split("_")
    return int(idx)
    
def stack(args):
    vertex, paths = args
    img = np.array([tifffile.imread(path) for path in sorted(paths, key=key)])
    sr = StackReg(StackReg.RIGID_BODY)
    registered = sr.register_transform_stack(img, reference="first")
    tifffile.imwrite(stacked_output / (vertex + ".tif"), registered)
    
inputs = list((experiment_base / "raw_imgs" / CHANNEL / "T1").glob("*/*.tif"))
groups = defaultdict(list)

for path in inputs:
    vertex = path.name.split("_")[0]
    groups[vertex].append(path)

try:
    from notebooks.config.dev import CPUS
except:
    from notebooks.config.default import CPUS

with Pool(CPUS // 2) as p:
    for _ in tqdm(p.imap(stack, groups.items()), total=len(groups)):
        continue

        
### PROCESS ###
        
stacks = list(stacked_output.glob("*.tif"))
output_path =  scratch_output / "motion_hilighted"
if output_path is not None and not output_path.exists():
    output_path.mkdir()

def proc(stack_path):
    vertex = stack_path.name.strip(".tif")
    try:
        stack = tifffile.imread(stack_path)
    except:
        return None
    motility = calc_motility(stack)
    if output_path is not None:
        rescaled = rescale_intensity(motility["motion_hilighted"], out_range=np.uint8)
        tifffile.imwrite(output_path / f"{vertex}.tif", rescaled)
    return {
        "vertex": vertex,
        "counts_per_frame": motility["counts_per_frame"],
        "frac_motile": motility["pct"]
    }

rows = []
with Pool(CPUS) as p:
    for motile_row in tqdm(p.imap(proc, stacks), total=len(stacks)):
        if motile_row is not None:
            rows.append(motile_row)

results_dir = experiment_base / "results"
if not results_dir.exists():
    results_dir.mkdir()
    
with open(results_dir / "mito_motility.csv", "w", newline="") as csvfile:
    writer = csv.DictWriter(csvfile, ["vertex", "counts_per_frame", "frac_motile"])
    writer.writeheader()
    writer.writerows(rows)